In [0]:
from pyspark.sql import SparkSession
from pyspark.sql.types import *

session = SparkSession.builder.appName('sparksession').getOrCreate()

### Dim_location

In [0]:
from pyspark.sql.functions import *

city_data = session.read.format('delta').load('dbfs:/user/hive/warehouse/cities')

In [0]:
city_data = city_data.select(col('id').alias('CityID'),
                col('lat').alias('Latitude'),
                col('lon').alias('Longitude'),
                col('name').alias('CityName'),
                col('country').alias('Country'))

In [0]:
session.sql('use weather')

Out[34]: DataFrame[]

In [0]:
city_data.write.option("overwriteSchema", "true")\
                .format('delta')\
                .mode('overwrite')\
                .save('dbfs:/databases/weather/dim_location')

### Dim_date

In [0]:
from pyspark.pandas import date_range

dt_col = spark.createDataFrame(date_range(start='2023-01-01', end='2023-12-31').to_numpy())
dt_col = dt_col.select("value",to_date("value").alias("Date")).drop("value")
dt_col = dt_col.withColumn("Date_id",monotonically_increasing_id())
dt_col = dt_col.select('Date_id','Date')
dt_col.show(10)

/databricks/spark/python/pyspark/pandas/namespace.py:1881: FutureWarning: Argument `closed` is deprecated in favor of `inclusive`.
  pd.date_range(


+-------+----------+
|Date_id|      Date|
+-------+----------+
|      0|2023-01-01|
|      1|2023-01-02|
|      2|2023-01-03|
|      3|2023-01-04|
|      4|2023-01-05|
|      5|2023-01-06|
|      6|2023-01-07|
|      7|2023-01-08|
|      8|2023-01-09|
|      9|2023-01-10|
+-------+----------+
only showing top 10 rows



In [0]:
dt_col = dt_col.withColumn('Month',  month(col('date')))\
               .withColumn('Year', year(col('date')))\
               .withColumn('DayOfMonth',  dayofmonth(col('date')))\
               .withColumn('DayOfWeek', dayofweek(col('date')))\
               .withColumn('DayOfYear', dayofyear(col('date')))\
               .withColumn('WeekofYear',  weekofyear(col('date')))\
               .withColumn('QuarterofYear', quarter(col('date')))
    

In [0]:
display(dt_col)

Date_id,Date,Month,Year,DayOfMonth,DayOfWeek,DayOfYear,WeekofYear,QuarterofYear
0,2023-01-01,1,2023,1,1,1,52,1
1,2023-01-02,1,2023,2,2,2,1,1
2,2023-01-03,1,2023,3,3,3,1,1
3,2023-01-04,1,2023,4,4,4,1,1
4,2023-01-05,1,2023,5,5,5,1,1
5,2023-01-06,1,2023,6,6,6,1,1
6,2023-01-07,1,2023,7,7,7,1,1
7,2023-01-08,1,2023,8,1,8,1,1
8,2023-01-09,1,2023,9,2,9,2,1
9,2023-01-10,1,2023,10,3,10,2,1


In [0]:

dt_col.write.option("overwriteSchema", "true")\
                .format('delta')\
                .mode('overwrite')\
                .save('dbfs:/databases/weather/dim_date')

### Dim_time

In [0]:

from pyspark.sql.functions import *


df = session.createDataFrame([('1997-02-28 00:00:00',)], ['Hour'])
df1 = df.select('Hour')
for i in range(24):
    df2 = df.select(col("Hour") + expr(str("INTERVAL " + str(i) + " HOURS"))).alias('Hour')
    df1 = df1.union(df2)
df1 = session.createDataFrame(df1.tail(df1.count()-1),df1.schema)
df1 = df1.select("Hour", (regexp_replace(col("Hour"),"1997-02-28 ","")).alias('newhour'))

df1 = df1.select(col('newhour').alias("Hour"))



In [0]:
dftime = session.range(0,23, step = 1).toDF("TimeID")
# dftime.show(2)

In [0]:
df1 = df1.withColumn("row_id",monotonically_increasing_id())
dftime = dftime.withColumn("row_id",monotonically_increasing_id())

data = dftime.join(df1,on="row_id").drop("row_id")

In [0]:
# data.show(100)

In [0]:

# df1.show()
# df1.count()
data.write.option("overwriteSchema", "true").format('delta')\
         .mode('overwrite')\
         .save('dbfs:/databases/weather/dim_time')



